In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

- California WARN data: https://edd.ca.gov/Jobs_and_Training/Layoff_Services_WARN.htm
- population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California Layoffs

For the purposes of this analysis, we are selecting 2020 records by isolating the year on the `Notice date` column.
By choosing `Notice date` we make sure that these layoffs were issued after the start of shelter in place for 2020, as some layoff notices can be issues much earlier the layoff date. Here, we are also doing a bit of data cleaning to standardize county names.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['County'] = cali_layoffs['County'].str.strip()
cali_layoffs.loc[~(cali_layoffs['County'].str.endswith('County')),'County'] = cali_layoffs['County'] + " County"
cali_layoffs['County'] = cali_layoffs['County'].str.replace('No County Name County', 'No Name')

## Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [6]:
cali_layoffs['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Temporary', 'Closure Temporary',
       'Layoff Permanent', 'Layoff Type Unknown', 'Closure Type Unknown',
       'Closure Unknown at this time', 'Layoff Unknown at this time'],
      dtype=object)

In [7]:
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.strip()
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.lower()
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure'].unique()

array(['closure permanent', 'layoff temporary', 'closure temporary',
       'layoff permanent', 'layoff type unknown', 'closure type unknown'],
      dtype=object)

In [8]:
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-8-3004cf2829a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5636,01/03/2020,03/06/2020,01/09/2020,"Medtronic, Inc.",Goleta,Santa Barbara County,33,layoff permanent,2020
5637,01/06/2020,03/07/2020,01/08/2020,"Edmunds.com, Inc.",Santa Monica,Los Angeles County,122,layoff permanent,2020
5638,01/06/2020,03/09/2020,01/06/2020,SunSelect Produce (California) Inc.,Tehachapi,Kern County,236,closure permanent,2020
5639,01/03/2020,03/04/2020,01/03/2020,"Aramark Services, Inc.",Anaheim,Orange County,9,closure permanent,2020
5640,01/03/2020,03/04/2020,01/03/2020,"Aramark Services, Inc.",Rancho SantaMargarita,Orange County,13,closure permanent,2020


In [9]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No Name,86.0,layoff permanent,2015
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No Name,49.0,closure permanent,2015
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No Name,11.0,closure permanent,2015
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No Name,18.0,layoff type unknown,2015
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No Name,50.0,closure permanent,2015


In [10]:
cali_layoffs['County'].unique()

array(['Los Angeles County', 'Orange County', 'San Bernardino County',
       'San Diego County', 'Sacramento County', 'Alameda County',
       'Santa Clara County', 'San Francisco County', 'Stanislaus County',
       'Contra Costa County', 'Riverside County', 'Solano County',
       'Ventura County', 'Monterey County', 'Placer County',
       'San Mateo County', 'Santa Barbara County', 'Fresno County',
       'Kings County', 'Kern County', 'Santa Cruz County',
       'Tehama County', 'Mendocino County', 'San Joaquin County',
       'Butte County', 'Sonoma County', 'Napa County', 'Calaveras County',
       'San Luis Obispo County', 'Siskiyou County', 'Yolo County',
       'Yuba County', 'Sutter County', 'Tulare County',
       'El Dorado County', 'Marin County', 'Mariposa County',
       'Imperial County', 'Shasta County', 'Madera County',
       'Del Norte County', 'Mono County', 'Merced County',
       'Humboldt County', 'San Benito County', 'Nevada County',
       'Tuolumne County',

In [11]:
years = ['2020', '2019']

## Extracting Only Necessary Records

Now that the data has been somewhat standardized, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [12]:
cali_2020_2019 = cali_layoffs[cali_layoffs["Year"].isin(years)]
cali_2020_2019

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,closure permanent,2020
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,closure permanent,2020
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,closure permanent,2020
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,layoff temporary,2020
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,layoff temporary,2020
...,...,...,...,...,...,...,...,...,...
6740,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",North Highlands,Sacramento County,15.0,closure permanent,2019
6741,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16.0,closure permanent,2019
6742,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84.0,closure permanent,2019
6743,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32.0,closure permanent,2019


## More Data Cleaning

A few records have the incorrect county, based on the information in the city column. These cities include: Sacramento, Hayward, Indian Wells, Cerritos, Vista, Roseville, City of Industry, Del Mar, Walnut Creek, and Los Gatos.

In [13]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

In [14]:
# Make a copy of this column instead

wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Sacramento') & (cali_2020_2019['County'] == 'Del Norte County')]
cali_2020_2019.loc[1793,'County'] = 'Sacramento County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Hayward') & (cali_2020_2019['County'] == 'Contra Costa County')]
cali_2020_2019.loc[5749,'County'] = 'Alameda County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Indian Wells') & (cali_2020_2019['County'] == 'San Benito County')]
cali_2020_2019.loc[5246,'County'] = 'Riverside County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Cerritos') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[3168,'County'] = 'Los Angeles County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Vista') & (cali_2020_2019['County'] == 'San Francisco County')]
cali_2020_2019.loc[5202,'County'] = 'San Diego County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Roseville') & (cali_2020_2019['County'] == 'Sacramento County')]
cali_2020_2019.loc[4748,'County'] = 'Placer County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'City of Industry') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[1772,'County'] = 'Los Angeles County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Del Mar') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[5190,'County'] = 'San Diego County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Walnut Creek') & (cali_2020_2019['County'] == 'Alameda County')]
cali_2020_2019.loc[4433,'County'] = 'Contra Costa County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Los Gatos') & (cali_2020_2019['County'] == 'Santa Cruz County')]
cali_2020_2019.loc[2648,'County'] = 'Santa Clara County'

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


## Exporting Standardized Data

Now that we have 2020 and 2019 data in an isolated dataframe, we can export it and clean the company names in Open Refine.
The data with clean company names will be re-read into the notebook on the line after the export. 

In [15]:
cali_2020_2019.to_csv('tobecleaned_california_warn.csv')

In [16]:
open_refine_clean = pd.read_csv('cleaned_california_warn.csv')

In [17]:
open_refine_clean.head()

,Column,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year
0,0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,closure permanent,2020
1,1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Gulfstream,Long Beach,Los Angeles County,608.0,closure permanent,2020
2,2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Renaissance Hotel,Newport Beach,Orange County,104.0,closure permanent,2020
3,3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,layoff temporary,2020
4,4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.","Newport Diversified, Inc.",El Cajon,San Diego County,91.0,layoff temporary,2020


In [18]:
open_refine_clean.drop(columns="Column", inplace=True)
open_refine_clean.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,closure permanent,2020
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Gulfstream,Long Beach,Los Angeles County,608.0,closure permanent,2020
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Renaissance Hotel,Newport Beach,Orange County,104.0,closure permanent,2020
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,layoff temporary,2020
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.","Newport Diversified, Inc.",El Cajon,San Diego County,91.0,layoff temporary,2020


## Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [19]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [20]:
cali_merge = pd.merge(open_refine_clean, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge['City 2'] = cali_merge['City'].str.lower()
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,closure permanent,2020,10039107,Los Angeles County,la mirada
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Gulfstream,Long Beach,Los Angeles County,608.0,closure permanent,2020,10039107,Los Angeles County,long beach
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Renaissance Hotel,Newport Beach,Orange County,104.0,closure permanent,2020,3175692,Orange County,newport beach
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,layoff temporary,2020,2180085,San Bernardino County,rancho cucamonga
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.","Newport Diversified, Inc.",El Cajon,San Diego County,91.0,layoff temporary,2020,3338330,San Diego County,el cajon


In [21]:
# pd.options.display.max_rows = 999
dupes = cali_merge[cali_merge[['Company 2', 'City 2', 'County', 'Employees', 'Year']].duplicated(keep=False)]
dupes.sort_values(by='Company 2', ascending=True)
dupes.head(10)

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
15,10/01/2020,12/01/2020,11/09/2020,Ace Beverage LLC,Ace Beverage LLC,Los Angeles,Los Angeles County,200.0,closure permanent,2020,10039107,Los Angeles County,los angeles
16,10/01/2020,12/11/2020,11/09/2020,Ace Beverage LLC,Ace Beverage LLC,Los Angeles,Los Angeles County,200.0,closure permanent,2020,10039107,Los Angeles County,los angeles
35,10/15/2020,02/22/2021,11/02/2020,Maritz Holdings Inc.,"Maritz Holdings, Inc.",Torrance,Los Angeles County,2.0,layoff permanent,2020,10039107,Los Angeles County,torrance
63,10/22/2020,11/01/2020,10/26/2020,"Gatan, Inc","Gatan, Inc",Pleasanton,Alameda County,105.0,layoff temporary,2020,1671329,Alameda County,pleasanton
72,05/22/2020,05/22/2020,10/26/2020,AIO Acquisition Inc.,AIO Acquisition Inc.,Ontario,San Bernardino County,5.0,layoff permanent,2020,2180085,San Bernardino County,ontario
79,08/19/2020,06/05/2020,10/23/2020,Rosa Mexicano,Rosa Mexicano,San Francisco,San Francisco County,76.0,closure permanent,2020,881549,San Francisco County,san francisco
86,10/15/2020,02/22/2021,10/22/2020,"Martiz Holdings, Inc.","Maritz Holdings, Inc.",Torrance,Los Angeles County,2.0,layoff permanent,2020,10039107,Los Angeles County,torrance
105,07/17/2020,07/17/2020,10/21/2020,"Centric Brands, Inc. dba Joe's Jeans Los Angel...",Centric Brands Inc.,Los Angeles,Los Angeles County,1.0,layoff permanent,2020,10039107,Los Angeles County,los angeles
152,07/21/2020,07/21/2020,10/12/2020,Electro Rent Corporation,Electro Rent,Canoga Park,Los Angeles County,25.0,layoff temporary,2020,10039107,Los Angeles County,canoga park
161,10/06/2020,10/09/2020,10/08/2020,ABM Aviation,"ABM Aviation, Inc.",Los Angeles,Los Angeles County,50.0,layoff permanent,2020,10039107,Los Angeles County,los angeles


In [22]:
## Creates DataFrame without the duplicate data, which means we will be undercounting. But all we need to do is state that.

cali_no_dupes = cali_merge[~cali_merge.isin(dupes)].dropna()

In [23]:
len(cali_no_dupes)

5958

In [24]:
## Number of Laid Off Workers 2019 #60719.0

cali_2019_info = cali_no_dupes[cali_no_dupes['Year'] == 2019.0]
cali_2019_info['Employees'].sum()

60719.0

In [25]:
# len(cali_2019_info)

In [26]:
# len(cali_2020_info)

In [27]:
## Number of Companies that laid people off 2019 #467

cali_2019_info['Company 2'].nunique()

467

In [28]:
## Number of Laid Off Workers 2020 #544407

cali_2020_info = cali_no_dupes[cali_no_dupes['Year'] == 2020.0]
SIP_perm_layoffs_closures_20 = cali_2020_info[cali_2020_info['Notice Date'] > '03/19/2020']
SIP_perm_layoffs_closures_20['Employees'].sum()

544407.0

In [29]:
## Number of Companies that laid people off 2020 #2600

SIP_perm_layoffs_closures_20['Company 2'].nunique()

2600

In [30]:
## Figures for closures/layoffs in 2019

figures_2019 = cali_2019_info.groupby(['Layoff/Closure']).sum()
figures_2019.reset_index()
figures_2019.drop(columns=['Year','Population'])

,Employees
Layoff/Closure,
closure permanent,27928.0
closure temporary,3233.0
closure type unknown,1505.0
layoff permanent,24185.0
layoff temporary,1913.0
layoff type unknown,1955.0


In [31]:
## Figures for closures/layoffs in 2020

figures_2020 = SIP_perm_layoffs_closures_20.groupby(['Layoff/Closure']).sum()
figures_2020.reset_index()
figures_2020.drop(columns=['Year','Population'])

,Employees
Layoff/Closure,
closure permanent,19316.0
closure temporary,58239.0
closure type unknown,2814.0
layoff permanent,94955.0
layoff temporary,346250.0
layoff type unknown,22833.0


In [32]:
layoffs = ['layoff permanent', 'layoff temporary', 'layoff type unknown']
closures = ['closure permanent', 'closure temporary', 'closure type unknown']

In [33]:
# of all layoff types (perm, temp, unknown) for 2019

layoffs_2019 = cali_2019_info[cali_2019_info['Layoff/Closure'].isin(layoffs)]
layoffs_2019['Employees'].sum()

28053.0

In [34]:
# of all closure types (perm, temp, unknown) for 2019

closures_2019 = cali_2019_info[cali_2019_info['Layoff/Closure'].isin(closures)]
closures_2019['Employees'].sum()

32666.0

In [35]:
# of all layoff types (perm, temp, unknown) for 2020

layoffs_2020 = SIP_perm_layoffs_closures_20[SIP_perm_layoffs_closures_20['Layoff/Closure'].isin(layoffs)]
layoffs_2020['Employees'].sum()

464038.0

In [36]:
# of all closure types (perm, temp, unknown) for 2020

closures_2020 = SIP_perm_layoffs_closures_20[SIP_perm_layoffs_closures_20['Layoff/Closure'].isin(closures)]
closures_2020['Employees'].sum()

80369.0

In [37]:
# check that they add up

layoffs_2020['Employees'].sum() + closures_2020['Employees'].sum()

544407.0

In [38]:
layoffs_2019['Employees'].sum() + closures_2019['Employees'].sum()

60719.0

## Calculating Total Layoffs + Closures Per Capita

In [39]:
# Calculations for 2019

employees_sum_2019 = cali_2019_info[['County','Employees']]
layoffs_sum_2019 = employees_sum_2019.groupby('County').agg('sum')
layoffs_sum_2019.reset_index()

,County,Employees
0,Alameda County,3741.0
1,Butte County,1874.0
2,Contra Costa County,1808.0
3,Fresno County,1189.0
4,Glenn County,143.0
5,Imperial County,44.0
6,Kern County,1523.0
7,Los Angeles County,14307.0
8,Mendocino County,80.0
9,Napa County,38.0


In [40]:
cali_info_and_sums_19 = pd.merge(cali_2019_info, layoffs_sum_2019, how="left", left_on="County", right_on="County")
cali_info_and_sums_19.rename(columns={'Employees_y':'Total Emp. Laid Off in County', 'Employees_x':'Employees'}, inplace=True)
cali_info_and_sums_19.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2,Total Emp. Laid Off in County
0,11/22/2019,01/21/2020,09/30/2020,"WeWork Companies, Inc.","WeWork Companies, Inc.",San Francisco,San Francisco County,326.0,layoff permanent,2019.0,881549.0,San Francisco County,san francisco,4666.0
1,04/29/2019,07/01/2019,03/03/2020,Kimberly-Clark Corporation,Kimberly-Clark Corporation,Fullerton,Orange County,204.0,closure permanent,2019.0,3175692.0,Orange County,fullerton,3831.0
2,05/02/2019,07/05/2019,02/25/2020,Alticor Inc.,Alticor Inc.,Buena Park,Orange County,45.0,layoff permanent,2019.0,3175692.0,Orange County,buena park,3831.0
3,12/31/2019,03/03/2020,01/02/2020,"Le Boulanger, Inc.","Le Boulanger, Inc.",Sunnyvale,Santa Clara County,143.0,closure permanent,2019.0,1927852.0,Santa Clara County,sunnyvale,7365.0
4,12/19/2019,02/13/2020,12/30/2019,NortonLifeLock Inc.,NortonLifeLock Inc.,Mountain View,Santa Clara County,19.0,layoff permanent,2019.0,1927852.0,Santa Clara County,mountain view,7365.0


In [41]:
cali_info_and_sums_19['Per Capita Laid Off 2019'] = (cali_info_and_sums_19['Total Emp. Laid Off in County']/cali_info_and_sums_19['Population']) * 100000
per_capita_laid_off = cali_info_and_sums_19[['County', 'Per Capita Laid Off 2019']]
per_capita_laid_off.groupby("County").max().sort_values(by='Per Capita Laid Off 2019', ascending=False)

,Per Capita Laid Off 2019
County,
Butte County,854.981614
San Francisco County,529.295592
Glenn County,503.645265
Santa Clara County,382.031401
Nevada County,244.599268
Alameda County,223.833847
San Mateo County,208.068899
Yolo County,200.907029
Placer County,189.792860


In [42]:
# Calculations for 2020

employees_sum_2020 = cali_2020_info[['County','Employees']]
layoffs_sum_2020 = employees_sum_2020.groupby('County').agg('sum')
layoffs_sum_2020.reset_index()

,County,Employees
0,Alameda County,31223.0
1,Amador County,159.0
2,Butte County,1617.0
3,Calaveras County,40.0
4,Colusa County,30.0
5,Contra Costa County,7174.0
6,Del Norte County,6.0
7,El Dorado County,819.0
8,Fresno County,5655.0
9,Humboldt County,227.0


In [43]:
cali_info_and_sums_20 = pd.merge(cali_2020_info, layoffs_sum_2020, how="left", left_on="County", right_on="County")
cali_info_and_sums_20.rename(columns={'Employees_y':'Total Emp. Laid Off in County', 'Employees_x':'Employees'}, inplace=True)
cali_info_and_sums_20.head()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2,Total Emp. Laid Off in County
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,closure permanent,2020.0,10039107.0,Los Angeles County,la mirada,182611.0
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Gulfstream,Long Beach,Los Angeles County,608.0,closure permanent,2020.0,10039107.0,Los Angeles County,long beach,182611.0
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Renaissance Hotel,Newport Beach,Orange County,104.0,closure permanent,2020.0,3175692.0,Orange County,newport beach,60139.0
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,layoff temporary,2020.0,2180085.0,San Bernardino County,rancho cucamonga,18269.0
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.","Newport Diversified, Inc.",El Cajon,San Diego County,91.0,layoff temporary,2020.0,3338330.0,San Diego County,el cajon,73950.0


In [44]:
cali_info_and_sums_20['Per Capita Laid Off 2020'] = (cali_info_and_sums_20['Total Emp. Laid Off in County']/cali_info_and_sums_20['Population']) * 100000
per_capita_laid_off_20 = cali_info_and_sums_20[['County', 'Per Capita Laid Off 2020']]
per_capita_laid_off_20.groupby("County").max().sort_values(by='Per Capita Laid Off 2020', ascending=False)

,Per Capita Laid Off 2020
County,
Mariposa County,13148.869383
Mono County,6085.571864
San Francisco County,5934.440400
Inyo County,2987.970508
San Diego County,2215.179446
San Mateo County,2152.306434
Napa County,1998.635149
Orange County,1893.728989
Alameda County,1868.154026


In [45]:
per_capitas = pd.merge(per_capita_laid_off_20, per_capita_laid_off, how='left', left_on="County", right_on="County")
per_capitas.groupby("County").max().sort_values(by='Per Capita Laid Off 2020', ascending=False)

,Per Capita Laid Off 2020,Per Capita Laid Off 2019
County,,
Mariposa County,13148.869383,NaN
Mono County,6085.571864,NaN
San Francisco County,5934.440400,529.295592
Inyo County,2987.970508,NaN
San Diego County,2215.179446,114.847843
San Mateo County,2152.306434,208.068899
Napa County,1998.635149,27.587409
Orange County,1893.728989,120.635125
Alameda County,1868.154026,223.833847


In [46]:
layoff_closure_list_perm = ['layoff permanent', 'closure permanent']
layoff_closure_list_temp = ['layoff temporary', 'closure temporary']
counties_list = ['Butte County', 'Santa Cruz County', 'Yolo County', 'Orange County', 'Los Angeles County', 'Humboldt County', 'Fresno County', 'Mariposa County']

## Perm Layoffs/Closures for 2020 and 2019

In [47]:
high_rent_burden_20_prm = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_prm = high_rent_burden_20_prm[high_rent_burden_20_prm['Layoff/Closure'].isin(layoff_closure_list_perm)]
high_rent_burden_20_prm = high_rent_burden_20_prm[['County','Layoff/Closure','Employees']]
high_rent_burden_20_prm = high_rent_burden_20_prm.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_20_prm.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,layoff permanent,570.0
Fresno County,closure permanent,980.0
Fresno County,layoff permanent,317.0
Los Angeles County,closure permanent,10145.0
Los Angeles County,layoff permanent,27912.0
Mariposa County,layoff permanent,45.0
Orange County,closure permanent,1387.0
Orange County,layoff permanent,12912.0
Santa Cruz County,closure permanent,447.0


In [48]:
high_rent_burden_19_prm = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_prm = high_rent_burden_19_prm[high_rent_burden_19_prm['Layoff/Closure'].isin(layoff_closure_list_perm)]
high_rent_burden_19_prm = high_rent_burden_19_prm[['County','Layoff/Closure','Employees']]
high_rent_burden_19_prm = high_rent_burden_19_prm.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_19_prm.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,closure permanent,501.0
Butte County,layoff permanent,42.0
Fresno County,closure permanent,1160.0
Fresno County,layoff permanent,10.0
Los Angeles County,closure permanent,7382.0
Los Angeles County,layoff permanent,5795.0
Orange County,closure permanent,2078.0
Orange County,layoff permanent,1501.0
Santa Cruz County,closure permanent,137.0


In [49]:
perm_job_loss = pd.merge(high_rent_burden_20_prm, high_rent_burden_19_prm, how='left', left_on=["County", "Layoff/Closure"], right_on=["County", "Layoff/Closure"])
perm_job_loss.rename(columns={'Employees_x':'Employees Laid Off 2020', 'Employees_y':'Employees Laid Off 2019'}, inplace=True)

## Temp Layoffs/Closures for 2020 and 2019

In [50]:
high_rent_burden_20_tmp = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_tmp = high_rent_burden_20_tmp[high_rent_burden_20_tmp['Layoff/Closure'].isin(layoff_closure_list_temp)]
high_rent_burden_20_tmp = high_rent_burden_20_tmp[['County','Layoff/Closure','Employees']]
high_rent_burden_20_tmp = high_rent_burden_20_tmp.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_20_tmp.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,layoff temporary,1002.0
Fresno County,closure temporary,882.0
Fresno County,layoff temporary,3476.0
Humboldt County,layoff temporary,227.0
Los Angeles County,closure temporary,19147.0
Los Angeles County,layoff temporary,116420.0
Mariposa County,closure temporary,343.0
Mariposa County,layoff temporary,1874.0
Orange County,closure temporary,6738.0


In [51]:
high_rent_burden_19_tmp = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_tmp = high_rent_burden_19_tmp[high_rent_burden_19_tmp['Layoff/Closure'].isin(layoff_closure_list_temp)]
high_rent_burden_19_tmp = high_rent_burden_19_tmp[['County','Layoff/Closure','Employees']]
high_rent_burden_19_tmp = high_rent_burden_19_tmp.groupby(['County','Layoff/Closure']).sum()
high_rent_burden_19_tmp.reset_index('Layoff/Closure')

,Layoff/Closure,Employees
County,,
Butte County,closure temporary,1331.0
Los Angeles County,closure temporary,326.0
Los Angeles County,layoff temporary,338.0
Santa Cruz County,layoff temporary,93.0


In [52]:
temp_job_loss = pd.merge(high_rent_burden_20_tmp, high_rent_burden_19_tmp, how='left', left_on=["County", "Layoff/Closure"], right_on=["County", "Layoff/Closure"])
temp_job_loss.rename(columns={'Employees_x':'Employees Laid Off 2020', 'Employees_y':'Employees Laid Off 2019'}, inplace=True)

## Temporary and Permanent Job Loss with 2020 and 2019 Comparisons

In [53]:
temp_job_loss

Employees Laid Off 2020  \
County             Layoff/Closure                               
Butte County       layoff temporary                    1002.0   
Fresno County      closure temporary                    882.0   
                   layoff temporary                    3476.0   
Humboldt County    layoff temporary                     227.0   
Los Angeles County closure temporary                  19147.0   
                   layoff temporary                  116420.0   
Mariposa County    closure temporary                    343.0   
                   layoff temporary                    1874.0   
Orange County      closure temporary                   6738.0   
                   layoff temporary                   36968.0   
Santa Cruz County  closure temporary                    241.0   
                   layoff temporary                    2702.0   
Yolo County        closure temporary                     24.0   
                   layoff temporary                     599.0   

                                      Employees Laid Off 2019  
County             Layoff/Closure                              
Butte County       layoff temporary                       NaN  
Fresno County      closure temporary                      NaN  
                   layoff temporary                       NaN  
Humboldt County    layoff temporary                       NaN  
Los Angeles County closure temporary                    326.0  
                   layoff temporary                     338.0  
Mariposa County    closure temporary                      NaN  
                   layoff temporary                       NaN  
Orange County      closure temporary                      NaN  
                   layoff temporary                       NaN  
Santa Cruz County  closure temporary                      NaN  
                   layoff temporary                      93.0  
Yolo County        closure temporary                      NaN  
                   layoff temporary                       NaN

In [54]:
perm_job_loss

Employees Laid Off 2020  \
County             Layoff/Closure                               
Butte County       layoff permanent                     570.0   
Fresno County      closure permanent                    980.0   
                   layoff permanent                     317.0   
Los Angeles County closure permanent                  10145.0   
                   layoff permanent                   27912.0   
Mariposa County    layoff permanent                      45.0   
Orange County      closure permanent                   1387.0   
                   layoff permanent                   12912.0   
Santa Cruz County  closure permanent                    447.0   
                   layoff permanent                     928.0   
Yolo County        closure permanent                     84.0   
                   layoff permanent                     457.0   

                                      Employees Laid Off 2019  
County             Layoff/Closure                              
Butte County       layoff permanent                      42.0  
Fresno County      closure permanent                   1160.0  
                   layoff permanent                      10.0  
Los Angeles County closure permanent                   7382.0  
                   layoff permanent                    5795.0  
Mariposa County    layoff permanent                       NaN  
Orange County      closure permanent                   2078.0  
                   layoff permanent                    1501.0  
Santa Cruz County  closure permanent                    137.0  
                   layoff permanent                       NaN  
Yolo County        closure permanent                    443.0  
                   layoff permanent                       NaN

In [55]:
high_rent_burden_19_prm = cali_2019_info[cali_2019_info['County'].isin(counties_list)]
high_rent_burden_19_prm = high_rent_burden_19_prm[high_rent_burden_19_prm['Layoff/Closure'].isin(layoff_closure_list_perm)]
perm_job_loss_total = high_rent_burden_19_prm[['County','Layoff/Closure','Employees']]
perm_job_loss_total = perm_job_loss_total.groupby(['County']).sum()
perm_job_loss_total

,Employees
County,
Butte County,543.0
Fresno County,1170.0
Los Angeles County,13177.0
Orange County,3579.0
Santa Cruz County,137.0
Yolo County,443.0


In [56]:
high_rent_burden_20_prm = cali_2020_info[cali_2020_info['County'].isin(counties_list)]
high_rent_burden_20_prm = high_rent_burden_20_prm[high_rent_burden_20_prm['Layoff/Closure'].isin(layoff_closure_list_perm)]
perm_job_loss_total_20 = high_rent_burden_20_prm[['County','Layoff/Closure','Employees']]
perm_job_loss_total_20 = perm_job_loss_total_20.groupby(['County']).sum()
perm_job_loss_total_20

,Employees
County,
Butte County,570.0
Fresno County,1297.0
Los Angeles County,38057.0
Mariposa County,45.0
Orange County,14299.0
Santa Cruz County,1375.0
Yolo County,541.0


In [57]:
sums_2020_2019 = perm_job_loss_total_20.merge(perm_job_loss_total, how="left", left_on="County", right_on="County")
sums_2020_2019.rename(columns={'Employees_x':'Permanent L/C Total 2020','Employees_y':'Permanent L/C Total 2019'}, inplace=True)
sums_2020_2019

,Permanent L/C Total 2020,Permanent L/C Total 2019
County,,
Butte County,570.0,543.0
Fresno County,1297.0,1170.0
Los Angeles County,38057.0,13177.0
Mariposa County,45.0,NaN
Orange County,14299.0,3579.0
Santa Cruz County,1375.0,137.0
Yolo County,541.0,443.0


In [58]:
#Fact Check for # of Layoffs in Mariposa County

Mariposa = cali_2020_info[cali_2020_info['County'] == 'Mariposa County']
Mariposa.groupby('County').sum() # Laid Off Employees totals 2,262

,Employees,Year,Population
County,,,
Mariposa County,2262.0,12120.0,103218.0


In [59]:
Mariposa.groupby('County').max() # Population totals 17,203

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,Year,Population,County Names,City 2
County,,,,,,,,,,,,
Mariposa County,07/30/2020,08/15/2020,08/03/2020,NatureBridge at Yosemite,NatureBridge,Yosemite National Park,1829.0,layoff temporary,2020.0,17203.0,Mariposa County,yosemite national park


In [60]:
Mariposa = cali_2020_info[cali_2020_info['County'] == 'Mariposa County']
Mariposa

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
975,07/30/2020,08/15/2020,08/03/2020,NatureBridge at Yosemite,NatureBridge,Yosemite National Park,Mariposa County,45.0,layoff permanent,2020.0,17203.0,Mariposa County,yosemite national park
1744,04/29/2020,03/22/2020,06/06/2020,El Portal Canyon Restaurant dba Miner'sRoadho...,El Portal Canyon Restaurant dba Miner'sRoadho...,Mariposa,Mariposa County,15.0,layoff temporary,2020.0,17203.0,Mariposa County,mariposa
1769,05/11/2020,03/26/2020,06/05/2020,El Portal Dining Inc dba Cedar House CanyonBar...,El Portal Dining Inc,El Portal,Mariposa County,9.0,layoff temporary,2020.0,17203.0,Mariposa County,el portal
1804,05/11/2020,03/26/2020,06/05/2020,El Portal Dining Inc dba Parkside Pizza,El Portal Dining Inc,El Portal,Mariposa County,21.0,layoff temporary,2020.0,17203.0,Mariposa County,el portal
1852,04/15/2020,03/19/2020,06/04/2020,Delaware North Companies Parks & Resorts atTen...,Delaware North Companies,Fish Camp,Mariposa County,343.0,closure temporary,2020.0,17203.0,Mariposa County,fish camp
3798,04/07/2020,03/13/2020,04/20/2020,Aramark - Yosemite,Aramark,Yosemite National Park,Mariposa County,1829.0,layoff temporary,2020.0,17203.0,Mariposa County,yosemite national park


In [61]:
# Fact Check that West Sacramento is most affected by Layoffs in Yolo County

Yolo = cali_2020_info[cali_2020_info['County'] == 'Yolo County']
Yolo = Yolo[Yolo['Year'] == 2020.0]
Yolo.groupby('City').sum()

,Employees,Year,Population
City,,,
Davis,290.0,28280.0,3087000.0
Madison,8.0,2020.0,220500.0
West Sacramento,704.0,26260.0,2866500.0
Winters,111.0,4040.0,441000.0
Woodland,197.0,12120.0,1323000.0


In [62]:
## Fact-Checking how many Layoffs Driven Performance Brands had in total for 2020.

Yolo = cali_2020_info[cali_2020_info['County'] == 'Yolo County']
sum_it = Yolo[Yolo['Company 2'] == 'Driven Performance Brands, Inc.']
sum_it.groupby('County').sum()

,Employees,Year,Population
County,,,
Yolo County,240.0,6060.0,661500.0


In [63]:
## Fact Check Santa Cruz County's permanent layoff figures

santa_cruz_20 = cali_2020_info[cali_2020_info['County'] == 'Santa Cruz County'] 
santa_cruz_20.groupby('County').max()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,Year,Population,County Names,City 2
County,,,,,,,,,,,,
Santa Cruz County,08/06/2020,09/01/2020,10/12/2020,Watsonville Community Hospital,Watsonville Community Hospital,Watsonville,1097.0,layoff type unknown,2020.0,273213.0,Santa Cruz County,watsonville


In [64]:
santa_cruz_20_perm = santa_cruz_20[santa_cruz_20['Layoff/Closure'] == 'layoff permanent']
santa_cruz_20_perm.groupby('County').max()

,Notice Date,Effective Date,Received Date,Company,Company 2,City,Employees,Layoff/Closure,Year,Population,County Names,City 2
County,,,,,,,,,,,,
Santa Cruz County,08/06/2020,09/01/2020,10/12/2020,Watsonville Community Hospital,Watsonville Community Hospital,Watsonville,265.0,layoff permanent,2020.0,273213.0,Santa Cruz County,watsonville


In [65]:
santa_cruz_20_perm[santa_cruz_20_perm['Employees'] == 265.0]

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
149,07/31/2020,09/01/2020,10/12/2020,"Mount Hermon Association, Inc.","Mount Hermon Association, Inc",Felton,Santa Cruz County,265.0,layoff permanent,2020.0,273213.0,Santa Cruz County,felton


In [66]:
## No Layoffs from Mount Hermon Association, so it seems like the layoffs for the company in 2020 are not a seasonal thing

santa_cruz_19 = cali_2019_info[cali_2019_info['County'] == 'Santa Cruz County']
santa_cruz_19

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
5769,10/09/2019,12/09/2019,10/11/2019,Chaminade Resort & Spa,Chaminade Resort & Spa,Santa Cruz,Santa Cruz County,93.0,layoff temporary,2019.0,273213.0,Santa Cruz County,santa cruz
5829,08/30/2019,10/31/2019,09/06/2019,"Coastline Support, Inc.","Coastline Support, Inc.",Santa Cruz,Santa Cruz County,55.0,closure permanent,2019.0,273213.0,Santa Cruz County,santa cruz
6150,04/12/2019,06/12/2019,04/12/2019,"Spawn Mate, Inc. dba Mushroom Farms","Spawn Mate, Inc. dba Mushroom Farms",Watsonville,Santa Cruz County,82.0,closure permanent,2019.0,273213.0,Santa Cruz County,watsonville


In [67]:
## Looking into Butte County, which had the highest rent burden

butte = cali_2020_info[cali_2020_info['County'] == 'Butte County']
butte

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
265,09/25/2020,08/19/2020,09/29/2020,"CSU, Chico","CSU, Chico",Chico,Butte County,446.0,layoff permanent,2020.0,219186.0,Butte County,chico
688,04/23/2020,04/24/2020,09/01/2020,"Lulu's Fashion Lounge, LLC","Lulu's Fashion Lounge, LLC",Chico,Butte County,78.0,layoff permanent,2020.0,219186.0,Butte County,chico
2033,04/21/2020,04/10/2020,05/27/2020,Associated Students of California StateUnivers...,Associated Students of California StateUnivers...,Chico,Butte County,660.0,layoff temporary,2020.0,219186.0,Butte County,chico
2572,03/23/2020,03/23/2020,05/14/2020,"Lulu's Fashion Lounge, LLC","Lulu's Fashion Lounge, LLC",Chico,Butte County,213.0,layoff temporary,2020.0,219186.0,Butte County,chico
3136,04/08/2020,03/17/2020,04/29/2020,Restec Contractors Inc. at CSU Chico,Restec Contractors Inc.,Chico,Butte County,8.0,layoff temporary,2020.0,219186.0,Butte County,chico
3241,04/17/2020,04/12/2020,04/28/2020,Dick's Sporting Goods Inc. at Chico Store#1089,Dick's Sporting Goods,Chico,Butte County,28.0,layoff temporary,2020.0,219186.0,Butte County,chico
3806,04/02/2020,03/24/2020,04/20/2020,Pancakes 1702 INC,"Pancakes, Inc - IHOP",Chico,Butte County,45.0,layoff type unknown,2020.0,219186.0,Butte County,chico
4209,03/26/2020,03/26/2020,04/15/2020,Cinemark USA Inc. dba Cinemark Chico,Cinemark USA Inc.,Chico,Butte County,46.0,layoff permanent,2020.0,219186.0,Butte County,chico
4435,03/30/2020,03/16/2020,04/13/2020,"Hupp Draft Services, Inc.","Hupp Draft Services, Inc.",Chico,Butte County,93.0,layoff temporary,2020.0,219186.0,Butte County,chico


In [68]:
butte_19 = cali_2019_info[cali_2019_info['County'] == 'Butte County']
butte_19 = butte_19[butte_19['Year'] == 2019.0]
butte_19

,Notice Date,Effective Date,Received Date,Company,Company 2,City,County,Employees,Layoff/Closure,Year,Population,County Names,City 2
5651,12/18/2019,02/01/2020,12/19/2019,Adventist Health Feather River Palliative Care,Adventist Health,Chico,Butte County,8.0,closure permanent,2019.0,219186.0,Butte County,chico
5741,10/28/2019,12/27/2019,10/29/2019,Lifetouch National School Studios Inc.,Lifetouch National School Studios Inc.,Chico,Butte County,320.0,closure permanent,2019.0,219186.0,Butte County,chico
5888,07/22/2019,09/20/2019,07/29/2019,"Milestone Technologies, Inc.","Milestone Technologies, Inc.",Chico,Butte County,148.0,closure permanent,2019.0,219186.0,Butte County,chico
5931,01/18/2019,03/17/2019,01/23/2019,Cerner Corporation,Cerner Corporation,Paradise,Butte County,42.0,layoff permanent,2019.0,219186.0,Butte County,paradise
5938,01/08/2019,02/05/2019,01/25/2019,Adventist Health Feather River,Adventist Health,Paradise,Butte County,1331.0,closure temporary,2019.0,219186.0,Butte County,paradise
6102,03/22/2019,05/24/2019,03/22/2019,Lifetouch National School Studios Inc.,Lifetouch National School Studios Inc.,Chico,Butte County,25.0,closure permanent,2019.0,219186.0,Butte County,chico
